In [13]:
import pandas as pd
import numpy as np
import os
import numpy
import matplotlib.pyplot as plt
import SimpleITK
import itertools
import sys
import torch
from torchvision import transforms
from PIL import Image
from matplotlib import cm
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
from torchvision import models

from pathlib import Path

from src.survival.survival_custom import SurvivalNet, get_survival_dataloaders, train_survivalnet

SOURCE_PATH = Path(os.getcwd()) / 'src'

if SOURCE_PATH not in sys.path:
    sys.path.append(SOURCE_PATH)

from src.extraction import (
    extract_images_in_survival_order,
    export_images_list_jpeg,
)

from src.plots import (
    plot_observation,
    plot_deeplab_mobile_predictions,
    plot_mobile_prediction_from_path
)

from src.deeplab_mobile.modelling import(
    select_images_input,
    get_deeplab_mobile_model,
    train_deeplab_mobile
)

from src.deeplab_mobile.segdataset import(
    get_mobile_dataloaders
)

from src.utils import(
    LOGS_FILE_PATH,
    TYPE_NAMES
)

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
data_path = Path(os.getcwd()) / 'data' / 'HGG'
survival = pd.read_csv(data_path.parent /'survival_data.csv')

dir_ids = survival['BraTS19ID']

In [15]:
t2, t1ce, t1, flair, seg = extract_images_in_survival_order(data_path, dir_ids)
images = [t2, t1ce, t1, flair, seg]

for i in range(len(TYPE_NAMES)):
    survival[TYPE_NAMES[i]] = images[i] 

In [16]:
modelname = 'flair_totalpipe_nighttrain.pt'
model = torch.load(Path(os.getcwd()) / 'models' / modelname)
model.eval()

DeepLabV3(
  (backbone): IntermediateLayerGetter(
    (0): ConvNormActivation(
      (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): Hardswish()
    )
    (1): InvertedResidual(
      (block): Sequential(
        (0): ConvNormActivation(
          (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (1): ConvNormActivation(
          (0): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        )
      )
    )
    (2): InvertedResidual(
      (block): Sequential(
        (0): ConvNormActivation(
          (0): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1),

In [17]:
flair_segout = []
for i in range(len(survival)):
    input_tensor = torch.tensor(survival['flair'][i]).expand(3, -1, -1).type(torch.ShortTensor).float().unsqueeze(0)
    with torch.no_grad():
        output = model(input_tensor)['out'][0][0]
    flair_segout.append(np.array(output))

survival['flair_seg'] = flair_segout

In [18]:
survival = survival.dropna().reset_index(drop=True)
survival['Survival'] = survival['Survival'].apply(lambda x: str(x))

to_drop = []
for i in range(len(survival)):
    if 'ALIVE' in survival['Survival'].loc[i]:
        to_drop.append(i)
        
        
survival.drop(to_drop, inplace=True)
print(len(survival))

103


In [19]:
survival['Survival'] = survival['Survival'].apply(lambda x: int(x))

In [20]:
net = SurvivalNet()

In [21]:
dataloaders = get_survival_dataloaders(survival['flair_seg'].values, survival['Survival'].values)

In [22]:
train_survivalnet(net, dataloaders, num_epochs=50)

Epoch 1/50
----------


100%|██████████| 8/8 [00:10<00:00,  1.33s/it]


Train Loss: 379369.4375


100%|██████████| 8/8 [00:01<00:00,  4.95it/s]


Test Loss: 354976.7500
{'epoch': 1, 'Train_loss': 379369.4375, 'Test_loss': 354976.75, 'Train_f1_score': 0.7777777777777778, 'Test_f1_score': [0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]}
{'epoch': 1, 'Train_loss': 379369.4375, 'Test_loss': 354976.75, 'Train_f1_score': 0.7777777777777778, 'Test_f1_score': 0.8888888888888888}
Epoch 2/50
----------


...

Test Loss: 69943.6875
{'epoch': 49, 'Train_loss': 73207.828125, 'Test_loss': 69943.6875, 'Train_f1_score': 0.8888888888888888, 'Test_f1_score': [0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]}
{'epoch': 49, 'Train_loss': 73207.828125, 'Test_loss': 69943.6875, 'Train_f1_score': 0.8888888888888888, 'Test_f1_score': 0.8888888888888888}
Epoch 50/50
----------


100%|██████████| 8/8 [00:07<00:00,  1.09it/s]


Train Loss: 80614.4062


100%|██████████| 8/8 [00:01<00:00,  5.12it/s]


Test Loss: 78451.2734
{'epoch': 50, 'Train_loss': 80614.40625, 'Test_loss': 78451.2734375, 'Train_f1_score': 0.8888888888888888, 'Test_f1_score': [0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]}
{'epoch': 50, 'Train_loss': 80614.40625, 'Test_loss': 78451.2734375, 'Train_f1_score': 0.8888888888888888, 'Test_f1_score': 0.8888888888888888}
Training complete in 8m 17s
Lowest Loss: 68152.781250


In [23]:
model_exp_name = "custom_made_survivalnet_final"
torch.save(net, str(Path(os.getcwd()) / "models" / (model_exp_name + ".pt")))

In [24]:
print('prediction', net(dataloaders['Test'].dataset[30]['image'].unsqueeze(0).float()))
print('target', dataloaders['Test'].dataset[30]['target'])

prediction tensor([[670.8815]], grad_fn=<AddmmBackward0>)
target 597
